In [1]:
from pathlib import Path

# Get the current working directory as a Path object
current_path = Path.cwd()
home_folder = 'evan_home'

# Traverse up the directory tree until you find the target folder
for parent in [current_path] + list(current_path.parents):
    if parent.name == home_folder:
        home_path = parent
        break
else:
    raise ValueError(f"Folder '{home_folder}' not found in the current working directory.")

print("Home Path:", home_path)
source_code_dir = home_path / 'Source_code'
dataset_dir = home_path / 'Dataset'


Home Path: c:\Users\evanlee\Documents\Bmi_NAS_evan\evan_home


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os

In [3]:
queue = ['CD4_T', 'DC', 'Mono', 'other_T']
len(queue)

4

In [4]:
# os.chdir('/home/jovyan/work/GitHub/EvanPys/Progress/PBMC_Hao_batch_noZ/Level1/tuning_result_cuda')
os.chdir(source_code_dir / 'PBMC_Hao_batch_noZ/Level1/tuning_result_cuda')

celltype = 'CD4_T'
with open(f'{celltype}_tuning.json') as f:
    tuning = json.load(f)
tuning.keys()

dict_keys(['Percentage', 'Prevalence', 'Feature_number', 'AUC', 'AUPR', 'MCC', 'Precision', 'F1 score', 'loss_history', 'error_history', 'log_lambda_range'])

In [5]:
# Find the index when 'Feature number' start to fall below 40
idx_40 = 0
for feature_no in tuning['Feature_number']:
    if feature_no < 40:
        print(idx_40)
        break
    idx_40 += 1

18


In [6]:
tuning['Feature_number']

[3287.0,
 2948.0,
 2613.0,
 2149.0,
 1568.0,
 1399.0,
 1343.0,
 917.0,
 805.0,
 706.0,
 444.0,
 504.0,
 277.0,
 191.0,
 138.0,
 121.0,
 64.0,
 50.0,
 35.0,
 24.0,
 20.0,
 14.0,
 8.0,
 4.0,
 5.0]

In [7]:
lmbd_40_features = np.exp(tuning['log_lambda_range'][idx_40])
lmbd_40_features

0.010000000000000004

In [5]:
# os.chdir('/home/jovyan/work/GitHub/EvanPys/Progress/PBMC_Hao_batch_noZ/Level1/tuning_result_cuda')
os.chdir(source_code_dir / 'PBMC_Hao_batch_noZ/Level1/tuning_result_cuda')

lmbd_40_features_dict = {}
for celltype in queue:
    print(celltype)
    with open(f'{celltype}_tuning.json') as f:
        tuning = json.load(f)
    
    # Find the index when 'Feature number' start to fall below 40
    idx_40 = 0
    for feature_no in tuning['Feature_number']:
        if feature_no < 40:
            # Get the previous index
            # idx_30 -= 1
            print(idx_40)
            break
        idx_40 += 1
    print('New number of features (<40):', tuning['Feature_number'][idx_40])

    lmbd_40_features = np.exp(tuning['log_lambda_range'][idx_40])
    lmbd_40_features_dict[celltype] = lmbd_40_features


CD4_T
18
New number of features (<40): 35.0
DC
14
New number of features (<40): 37.0
Mono
16
New number of features (<40): 39.0
other_T
16
New number of features (<40): 37.0


In [10]:
lmbd_40_features_dict

{'CD4_T': 0.010000000000000004,
 'DC': 0.0021544346900318843,
 'Mono': 0.004641588833612781,
 'other_T': 0.004641588833612781}

In [ ]:
# write to json
# os.chdir('/home/jovyan/work/GitHub/EvanPys/Progress/PBMC_Hao_batch_noZ/Level1/large_lambda')
os.chdir(source_code_dir / 'PBMC_Hao_batch_noZ/Level1/large_lambda')
with open('L1_lambda_at_40_features.json', 'w') as f:
    json.dump(lmbd_40_features_dict, f)